In [51]:
import os
import json
from urllib.parse import urlparse, urljoin
import requests
import geopandas as gpd
from shapely.geometry import box, shape
from tqdm import tqdm

# Get list of datasets from OT API query

In [11]:
from differencing_functions import DataAccess, OpenTopographyQuery, GetDEMs

## Load API key

In [8]:
# After setting the environment variable, access your API key in this notebook.
API_Key = os.getenv('OPENTOPO_ADMIN_ENTERPRISE_API_KEY')

# If your API key is not set, you can set it here.
# API_Key = "your_api_key_here"

if API_Key is not None:
    print("API Key loaded successfully!")
else:
    print("Failed to load API Key.")

API Key loaded successfully!


<h3 id="Option-3-Upload-File">Define bounds using an uploaded file</h3>

In [9]:
shapefile_path = "/Users/cassandrabrigham/ASU Dropbox/Cassandra Brigham/Mac/Documents/POSTDOC/Offset mapping - SCEC/GIS/san_jacinto.shp"

da = DataAccess()

da.define_bounds_from_file(shapefile_path, target_crs = 'EPSG:4326')

{'south': 32.79130944963459,
 'west': -116.93032727942875,
 'north': 33.72062768978766,
 'east': -115.57803503695636,
 'polygon_wkt': ['-116.93032727942875, 33.67457519663469, -116.87254408082845, 33.72062768978766, -116.80774049361317, 33.70715146646199, -116.49830336466015, 33.53446917230551, -116.42053906000177, 33.480435035977344, -116.16294480082101, 33.33031374927006, -115.81948578857995, 33.156879269002346, -115.68501834510822, 33.048308147401876, -115.57803503695636, 32.86279561564286, -115.57938537217136, 32.83670416059145, -115.64825246813542, 32.81968386172708, -115.69821487108975, 32.79130944963459, -116.18568588369826, 33.121025783963546, -116.89931103811193, 33.63786514990443, -116.93032727942875, 33.67457519663469']}

<h3 id="Use-OT-Catalog - Single DEM"> Use OT Catalog To Find Datasets</h3>


In [18]:
# Build a DataAccess and call the wrapper:
da = DataAccess()
da.define_bounds_from_file(shapefile_path, target_crs="EPSG:4326")

otq, catalog_df = GetDEMs.query_single_dem(
    da,
    product_format="PointCloud",
    include_federated=True,
    detail=False,
    save_as="results.json",   
)

catalog_df

,Name,ID type,Data Source,Property ID,Horizontal EPSG,Vertical Coordinates,Clean Name
0,B4 Project - Southern San Andreas and San Jaci...,opentopoID,ot,OTLAS.032018.32611.1,32611,Ellipsoid,B4_Project_Southern_San_Andreas_and_San_Jacint...
1,2010 Salton Sea Lidar Collection,opentopoID,ot,OTLAS.032012.26911.2,26911,NAVD88 (GEOID 09),2010_Salton_Sea_Lidar_Collection
2,B4 Project - Southern San Andreas and San Jaci...,opentopoID,ot,OTLAS.032006.32611.1,32611,Ellipsoid,B4_Project_Southern_San_Andreas_and_San_Jacint...
3,CA SaltonSea EarthMRI 3 D21,USGS_3DEP_ID,usgs,CA_SaltonSea_EarthMRI_3_D21,3857,NAVD88 height - Geoid18 (Meters),CA_SaltonSea_EarthMRI_3_D21
4,CA SaltonSea EarthMRI 1 2021,USGS_3DEP_ID,usgs,CA_SaltonSea_EarthMRI_1_2021,3857,NAVD88 height - Geoid18 (Meters),CA_SaltonSea_EarthMRI_1_2021
5,USGS LPC CA SoCal Wildfires B1 2018 LAS 2019,USGS_3DEP_ID,usgs,USGS_LPC_CA_SoCal_Wildfires_B1_2018_LAS_2019,3857,NAVD88 height - Geoid12B (metre),USGS_LPC_CA_SoCal_Wildfires_B1_2018_LAS_2019
6,USGS LPC CA SoCAL Wildfires TL 2018 LAS 2019,USGS_3DEP_ID,usgs,USGS_LPC_CA_SoCAL_Wildfires_TL_2018_LAS_2019,3857,NAVD88 height - Geoid12B (metre),USGS_LPC_CA_SoCAL_Wildfires_TL_2018_LAS_2019
7,CA SaltonSea 2010,USGS_3DEP_ID,usgs,CA_SaltonSea_2010,3857,NAVD88 - Geoid09 (Meters),CA_SaltonSea_2010
8,USGS LPC CA E SanDiegoCo 2016 LAS 2017,USGS_3DEP_ID,usgs,USGS_LPC_CA_E_SanDiegoCo_2016_LAS_2017,3857,NAVD88 height (ftUS),USGS_LPC_CA_E_SanDiegoCo_2016_LAS_2017


In [19]:
selected_dataset_index = 3
print(f"Selected dataset: {catalog_df['Name'][selected_dataset_index]}")

Selected dataset: CA SaltonSea EarthMRI 3 D21


## Parameters

In [70]:
SHAPEFILE    = shapefile_path
CATALOG_URL  = "https://usgs-lidar-stac.s3-us-west-2.amazonaws.com/ept/catalog.json"
PREFIX       = catalog_df['Property ID'][selected_dataset_index]
BUCKET_DOMAIN = "s3-us-west-2.amazonaws.com"
BUCKET       = "usgs-lidar-public"
EPT_BASE      = f"https://{BUCKET_DOMAIN}/{BUCKET}/{PREFIX}/"
OUTPUT_LIST  = "nodes_to_download.txt"
TARGET_EPSG    = 3857

## 1. Build nodes_to_download.txt by traversing ept-hierarchy

### Load & union polygon (WGS84)

In [71]:
gdf = gpd.read_file(SHAPEFILE)
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)
# project into dataset CRS
gdf_proj = gdf.to_crs(epsg=TARGET_EPSG)
region   = gdf_proj.geometry.union_all()

### Read top‐level ept.json for bounds

In [77]:
# 2️⃣ Fetch ept.json bounds (in EPSG:3857)
resp = requests.get(urljoin(EPT_BASE, "ept.json"))
resp.raise_for_status()
meta = resp.json()
minx, miny, _, maxx, maxy, _ = meta["bounds"]

# Optional sanity check
print(f"Dataset bounds (EPSG:{TARGET_EPSG}):", (minx, miny, maxx, maxy))
print(f"Region proj bounds:            ", region.bounds)
print("Root intersects region?", region.intersects(box(minx, miny, maxx, maxy)))

Dataset bounds (EPSG:3857): (-13003569, 3844743, -12816559, 4031753)
Region proj bounds:             (-13016624.491036834, 3867636.4310987936, -12866088.007201115, 3991350.555281892)
Root intersects region? True


### Fetch the root hierarchy mapping

In [79]:
r = requests.get(urljoin(EPT_BASE, "ept-hierarchy/0-0-0-0.json"))
r.raise_for_status()
mapping = r.json()  # {nodeID: pointCount or -1 if split}
node_ids = list(mapping.keys())
print(f"Total nodes in hierarchy mapping: {len(node_ids)}")

Total nodes in hierarchy mapping: 28972


### Identify leaf IDs (no deeper descendent in mapping)

In [80]:
nodes = []
for nid in node_ids:
    d, x, y, z = map(int, nid.split("-"))
    nodes.append({"id": nid, "d": d, "x": x, "y": y, "z": z})

leaf_nodes = []
for node in nodes:
    nd, nx, ny, nz = node['d'], node['x'], node['y'], node['z']
    # look for any child at depth nd+1 that maps back to this node
    has_child = any(
        (other['d'] == nd + 1) and
        (other['x'] // 2 == nx) and
        (other['y'] // 2 == ny) and
        (other['z'] // 2 == nz)
        for other in nodes
    )
    if not has_child:
        leaf_nodes.append(node)

print(f"Detected {len(leaf_nodes)} leaf nodes")

Detected 21549 leaf nodes


### Spatial filter leaves by intersection

In [81]:
to_download = []
for node in tqdm(leaf_nodes, desc="Filtering leaves"):
    nd, nx, ny = node['d'], node['x'], node['y']
    # compute 2D footprint
    tile_w = (maxx - minx) / (2 ** nd)
    tile_h = (maxy - miny) / (2 ** nd)
    tx0 = minx + nx * tile_w; ty0 = miny + ny * tile_h
    tx1 = tx0 + tile_w;   ty1 = ty0 + tile_h
    if region.intersects(box(tx0, ty0, tx1, ty1)):
        to_download.append(f"{PREFIX}/ept-data/{node['id']}.laz")

print(f"{len(to_download)} leaf tiles intersect your region")

Filtering leaves: 100%|██████████| 21549/21549 [00:00<00:00, 76129.04it/s]

5425 leaf tiles intersect your region


### Write out full S3 URIs

In [83]:
os.makedirs(os.path.dirname(OUTPUT_LIST) or ".", exist_ok=True)
with open(OUTPUT_LIST, "w") as f:
    for key in sorted(set(to_download)):
        f.write(f"s3://{BUCKET}/{key}\n")
print(f"Wrote {len(to_download)} lines to {OUTPUT_LIST}")

Wrote 5425 lines to nodes_to_download.txt
